In [1]:
import pandas as pd
import ast

file_path = 'datasources/archive/NYC_Restaurants_rawdata.csv'
main_df = pd.read_csv(file_path)
main_df['coordinates'] = main_df['coordinates'].apply(ast.literal_eval)
main_df['location'] = main_df['location'].apply(ast.literal_eval)

main_df['latitude'] = main_df['coordinates'].apply(lambda x: x['latitude'] if x else None)
main_df['longitude'] = main_df['coordinates'].apply(lambda x: x['longitude'] if x else None)
main_df['address'] = main_df['location'].apply(lambda x: x['address1'] if x else None)
main_df['city'] = main_df['location'].apply(lambda x: x['city'] if x else None)
main_df['zip_code'] = main_df['location'].apply(lambda x: x['zip_code'] if x else None)
main_df['state'] = main_df['location'].apply(lambda x: x['state'] if x else None)

columns_to_drop = ['alias','categories','transactions','distance','image_url', 'is_closed', 'display_phone', 'coordinates', 'location']
main_df.drop(columns=columns_to_drop, inplace=True)

main_df['phone'] = main_df['phone'].astype(str).str.replace('+', '', regex=False).apply(lambda x: x[1:] if x.startswith('1') else x)
main_df['phone'] = main_df['phone'].apply(lambda x: x[:-2] if x.endswith('.0') else x)

main_df['street_name'] = main_df['address'].str.split().str[1:].str.join(' ')
main_df['street_name'] = main_df['street_name'].str.upper()

main_df['Name_UPPER'] = main_df['name'].str.upper()

main_df.rename(columns={
    'id': 'ID',
    'name': 'Name',
    'url': 'URL',
    'review_count': 'Review_Count',
    'rating': 'Rating',
    'price': 'Price',
    'phone': 'Phone',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'address': 'Address',
    'city': 'City',
    'zip_code': 'Zip',
    'state': 'State',
    'street_name': 'Street'
}, inplace=True)

column_order = ['Name','Address', 'City', 'Zip', 'State', 'Longitude', 'Latitude', 'Review_Count', 'Rating', 'Price', 'URL', 'ID', 'Phone', 'Street', 'Name_UPPER']
main_df = main_df[column_order]

main_df.head()

,Name,Address,City,Zip,State,Longitude,Latitude,Review_Count,Rating,Price,URL,ID,Phone,Street,Name_UPPER
0,Thursday Kitchen,424 E 9th St,New York,10009,NY,-73.983730,40.727610,1772,4.5,$$,https://www.yelp.com/biz/thursday-kitchen-new-...,fVbUVAiLiGgLA_nxBFxyww,nan,E 9TH ST,THURSDAY KITCHEN
1,Time Out Market New York,55 Water St,Brooklyn,11201,NY,-73.992146,40.703429,516,4.0,$$,https://www.yelp.com/biz/time-out-market-new-y...,MGd6HFEq1ALD58XWNviSXw,9178104855,WATER ST,TIME OUT MARKET NEW YORK
2,ARIARI,119 1st Ave,New York,10003,NY,-73.985831,40.727284,197,4.5,NaN,https://www.yelp.com/biz/ariari-new-york?adjus...,nMyot99PkX5iYCdxHoRJqQ,6464227466,1ST AVE,ARIARI
3,Salma,351 E 12th St,New York,10003,NY,-73.983722,40.730370,167,4.5,$$,https://www.yelp.com/biz/salma-new-york?adjust...,IA3EQ0Ilx0yI7dNYnq-YGQ,9172614900,E 12TH ST,SALMA
4,Kalye,251 Broome St,New York,10002,NY,-73.989990,40.717900,96,4.5,$$,https://www.yelp.com/biz/kalye-new-york?adjust...,hLXe3RVRK39VUSPdvBjFEA,6464227267,BROOME ST,KALYE


In [2]:
file_path = 'datasources/archive/NYC_Restaurants_rawdata.csv'
raw_df = pd.read_csv(file_path)

raw_df['categories'] = raw_df['categories'].apply(ast.literal_eval)

category_rows = []

for idx, row in raw_df.iterrows():
    business_id = row['id']
    categories = row['categories']
    for category in categories:
        alias = category['alias']
        title = category['title']
        category_rows.append([business_id, alias, title])

categories_df = pd.DataFrame(category_rows, columns=['id', 'category_alias', 'category_title'])

categories_df.head()

,id,category_alias,category_title
0,fVbUVAiLiGgLA_nxBFxyww,korean,Korean
1,fVbUVAiLiGgLA_nxBFxyww,newamerican,American (New)
2,fVbUVAiLiGgLA_nxBFxyww,tapasmallplates,Tapas/Small Plates
3,MGd6HFEq1ALD58XWNviSXw,food_court,Food Court
4,nMyot99PkX5iYCdxHoRJqQ,korean,Korean


In [3]:
unique_categories = categories_df['category_title'].unique()
unique_categories_df = pd.DataFrame(unique_categories, columns=['Unique Categories'])
unique_categories_df

,Unique Categories
0,Korean
1,American (New)
2,Tapas/Small Plates
3,Food Court
4,Gastropubs
...,...
222,Social Clubs
223,Cooking Classes
224,Golf Lessons
225,Olive Oil


In [4]:
categories_df.to_csv('datasources/modified/yelp_NYC_Restaurants_categories.csv', index=False)
main_df.to_csv('datasources/modified/yelp_NYC_Restaurants.csv', index=False)
unique_categories_df.to_csv('datasources/modified/yelp_uniquecategorieslist.csv', index=False)